In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import os,sys
import utilities
import matplotlib.pyplot as plt
from numpy import array
from multiprocessing import Pool
import multiprocessing as mp
from consts import sensor_lat as lat
from consts import sensor_log as log
from consts import valid_sensor_res_10 as valid_sensor_list
from consts import valid_sensor_res_10_old as valid_sensor_list_old
from consts import sensor_day_seconds as day_seconds
from consts import sensor_week_seconds as week_seconds 
from consts import sensor_start_time_unix as start_time
from consts import sensor_end_time_unix as end_time
from consts import sensor_epoch_length as epoch_length
from consts import sensor_optimal_time_interval as optimal_time_interval
from consts import sensor_loc_pic as loc_pic
from consts import sensor_drop_list as drop_list
import consts
import time
import pickle
import copy
import h5py

In [3]:
time_stamp = np.arange(start_time,end_time+optimal_time_interval,optimal_time_interval)
unix_column = 'Time since the epoch [s]'
idd = 'Station ID'
a_temperature = 'Ambient Temperature'
s_temperature = 'Surface Temperature'

In [6]:
df = pd.read_hdf('optimal_period.h5','optimal_df')

In [7]:
df

,Station ID,Year,Month,Day,Hour,Minute,Second,Time since the epoch [s],Sequence Number,Ambient Temperature,Surface Temperature,Relative Humidity [%]
319633,8,2007,3,1,0,0,27,1172703627,35250,8.10,7.350,81.956
319634,8,2007,3,1,0,0,56,1172703656,35251,8.18,7.350,81.257
319635,8,2007,3,1,0,1,25,1172703685,35252,8.20,7.537,81.141
319636,8,2007,3,1,0,1,55,1172703715,35253,8.20,7.412,81.788
319637,8,2007,3,1,0,2,24,1172703744,35254,8.12,7.350,83.592
...,...,...,...,...,...,...,...,...,...,...,...,...
777546,121,2007,3,7,17,30,33,1173285033,41669,10.23,11.350,67.417
777547,121,2007,3,7,17,30,43,1173285043,41670,10.26,11.412,66.656
777548,121,2007,3,7,17,30,52,1173285052,41671,10.27,11.350,66.795
777549,121,2007,3,7,17,31,2,1173285062,41672,10.28,11.225,66.906


In [5]:
df.to_hdf('optimal_period.h5',key='optimal_df')

In [15]:
data_index = ['a_temperature_mean','a_temperature_std','a_temperature_array','sub_df_list_optimal_index','time_stamp',]

In [16]:
with h5py.File('dataset.hdf5','r') as f:
     a_temperature_mean = f[data_index[0]][:]
     a_temperature_std = f[data_index[1]][:]
     a_temperature_array = f[data_index[2]][:]
     sub_df_list_optimal_index = f[data_index[3]][:]
     time_stamp = f[data_index[4]][:]

In [10]:
df_dict = {'rb_optimal':5959,'optimal':5959,'df_list':20160}

In [23]:
def dataset_write(data_list,key,df_dict=df_dict):
    filename = 'ds_'+key+'.h5'
    store = pd.HDFStore(filename)
    for i,item in enumerate(data_list):
        store[key+'_'+str(i)] = item
    store.close()
    return 1

In [11]:
def dataset_read(key,df_dict=df_dict):
    filename = 'ds_'+key+'.h5'
    return_list = []
    store = pd.HDFStore(filename)
    for i in range(df_dict[key]):
        return_list.append(store[key+'_'+str(i)])
    store.close()
    return return_list

In [ ]:
dataset_write(rb_sub_df_list_optimal,'rb_optimal')

In [ ]:
dataset_write(sub_df_list_optimal,'optimal')

In [ ]:
dataset_write(sub_df_list,'df_list')

In [13]:
rb_sub_df_list_optimal = dataset_read('rb_optimal')

30453            39295                10.31   
 147579                1172830461            19322                 9.74   
 192833                1172830445            19337                10.22   
 731710                1172830449            60657                15.50   
 
         Surface Temperature  Relative Humidity [%]  
 323934                9.475                 65.287  
 287399               13.225                 66.064  
 329176                7.912                 69.427  
 326465               14.975                 62.941  
 296281               10.350                 67.943  
 330212               13.850                 65.825  
 344725               15.600                 67.542  
 298217               14.225                 63.646  
 308271                8.787                 70.077  
 328830                8.100                 67.318  
 328739               25.225                 65.467  
 291245                9.850                 67.165  
 326294               11